In [453]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta
import win32com.client as win32
import teradatasql
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pygwalker as pyg
from pandas.tseries.offsets import DateOffset

Step 1. Данные К-Ц с ресурса

In [454]:
df_kc = pd.read_excel(r'\\corp.tele2.ru\zrfolders\InternalReportCC\!Отчетность Центра поддержки\InHouse_NPS\InHouse_NPS.xlsx')
#df_kc = pd.read_excel('InHouse_NPS.xlsx')
#for i in df_kc.columns:
#    print(f" '{i}', ")

In [494]:
result = [] 
directory = '//corp.tele2.ru/msfolders/inhouse_tp'
 
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.csv'):
            result.append(os.path.join(root, file))
result  
#мапп данных 
#folder_path = '//corp.tele2.ru/msfolders/inhouse_tp\\221216'
#file_list = glob.glob(folder_path + "/*.csv")
main_dataframe = pd.DataFrame(pd.read_table(result[0],   delimiter= ";", encoding="cp1251", on_bad_lines='skip', header=None))
for i in range(1,len(result)):
    data = pd.read_table(result[i], delimiter= ";", encoding="cp1251", on_bad_lines='skip', header=None)
    df = pd.DataFrame(data)
    main_dataframe = pd.concat([main_dataframe, df], axis = 0)  
main_dataframe.reset_index(drop= True , inplace= True)  

#обработка данных
main_dataframe.columns = [
'WORKLIST_ID', 'CLIENT_ID', 'PHONE1','PHONE2','PHONE3','MSISDN','TARIFF_PLAN','SCRIPT_ID','P_INVOICE','PA_INVOICE','CUST1', 'CUST2','CUST3',
'CUST4','CUST5','CUST6','NPS','REP_DTE' ,'REGION',  'CONT_POINT','QW1', 'QW2','QW3','QW4','QW5','COM5','QW6','COM6','COMP_PACK','CUST_COMM', 
'VOICE_1','VOICE_2','VOICE_3','VOICE_COMM', 'GPRS_1' , 'GPRS_2' , 'GPRS_3', 'GPRS_COMM' ,'CC','CC_COMM' ,'OFFICE'  ,'OFFICE_COMM','ROAMING',
'ROAMING_COMM' ,'DISHONESTY' ,  'DISHONESTY_COMM' , 'APP' , 'APP_COMM' ,'OBSESSION' ,'OBSESSION_COMM' ,'TARIFF' ,'TARIFF_COMM' ,'ERR_DETR' ,
'ERR_DETR_COMM','OTHER','CALL_RESULT','TREATMENTCODE' ,'user', 'date_call','OBJ3','OBJ4','OBJ5','OBJ6','OBJ7','OBJ8','OBJ9','OBJ10','OBJ11',
'OBJ12','OBJ13','OBJ14','OBJ15','OBJ16','OBJ17' ,'OBJ18' ,'OBJ19','OBJ20','OBJ21' , 'OBJ22' , 'OBJ23' , 'OBJ24','OBJ25' , 'OBJ26' , 'OBJ27',
'OBJ28', 'OBJ29', 'OBJ30' ,  'OBJ31','OBJ32' , 'OBJ33','OBJ34','OBJ35','OBJ36','OBJ37' , 'OBJ38','OBJ39' , 'OBJ40','OBJ41','OBJ42', 'OBJ43',
'OBJ44', 'OBJ45','OBJ46','OBJ47' ,'OBJ48' , 'OBJ49'  , 'OBJ50','OBJ51','OBJ52' ,'OBJ53' , 'OBJ54' ,'OBJ55', 'OBJ56','OBJ57','OBJ58','OBJ59',
'OBJ60','OBJ61', 'OBJ62', 'OBJ62','OBJ63','OBJ64','OBJ65','OBJ66','OBJ67','OBJ68','OBJ69','OBJ70', 'OBJ71'
]


main_dataframe['date_call'] = pd.to_datetime(main_dataframe['date_call'])
main_dataframe.dropna(subset=['MSISDN'], inplace= True) 

main_dataframe['year'] = pd.DatetimeIndex(main_dataframe['date_call']).year
main_dataframe['month'] = pd.DatetimeIndex(main_dataframe['date_call']).month
main_dataframe['week'] = pd.to_datetime(main_dataframe['date_call']).dt.isocalendar().week
#main_dataframe = main_dataframe.dropna (subset=['year'])
#main_dataframe['year'] = main_dataframe['year'].astype(int)
main_dataframe = main_dataframe[[
'MSISDN', 'NPS', 'user', 'REP_DTE', 'week',  'month', 'year', 'REGION', 
'OBJ71', 
'CONT_POINT', 'QW1',   'QW2'  , 'QW3','QW4', 'QW5' ,'COM5', 'QW6', 'COM6', 'COMP_PACK',
'CUST_COMM', 'VOICE_1', 'VOICE_2', 'VOICE_3', 'VOICE_COMM', 'GPRS_1' , 'GPRS_2','GPRS_3', 'GPRS_COMM',
'CC', 'CC_COMM' , 'OFFICE' , 'OFFICE_COMM', 'ROAMING', 'ROAMING_COMM', 'DISHONESTY', 'DISHONESTY_COMM',
'APP' , 'APP_COMM', 'OBSESSION', 'OBSESSION_COMM', 'TARIFF', 'TARIFF_COMM', 'ERR_DETR', 'ERR_DETR_COMM',
'OTHER','CALL_RESULT', 'date_call', 
'OBJ55', 'OBJ56','OBJ57','OBJ58','OBJ59',
'OBJ60','OBJ61', 'OBJ62', 'OBJ63','OBJ64','OBJ65','OBJ66','OBJ67','OBJ68','OBJ69','OBJ70']]

main_dataframe.columns = ['custnum', 'nps', 
'user', 'date', 
'week', 'month', 
'year', 'region', 
'macro', 'cont_point', 
'qw1', 'qw2', 
'qw3', 'qw4', 
'qw5', 'com5', 
'qw6', 'com6', 
'comp_pack', 'cust_comm', 
'voice_1', 'voice_2', 
'voice_3', 'voice_comm', 
'gprs_1', 'gprs_2', 
'gprs_3', 'gprs_comm', 
'cc', 'cc_comm', 
'office', 'office_comm', 
'roaming', 'roaming_comm', 
'dishonesty', 'dishonesty_comm', 
'app', 'app_comm', 
'obsession', 'obsession_comm', 
'tarif', 'tarif_comm', 
'err_detr', 'err_detr_comm', 
'other', 'call_result', 
'date_call', 'code', 
'mark1', 'mark2', 
'mark3', 'mark4', 
'mark5', 'Тип оффера', 
'idinhouseservices', 'entryid', 
'service', 'sd', 
'Location', 'Coordinate', 
'description', 'Заявка', 
'Оффер', 'Реальная ТК']


In [495]:
main_dataframe.reset_index(inplace=True, drop=True)
df_kc.reset_index(inplace=True, drop=True)


In [ ]:
df_svod = pd.concat([df_kc, main_dataframe], ignore_index=True)
df_svod

In [497]:
df_svod.to_excel('test.xlsx')

In [433]:
df_kc = df_kc.reindex(columns=['custnum', 'nps', 'user', 'date', 'week', 'month', 'year', 'region', 'macro', 'cont_point',  'cust_comm', 'voice_1',
                               'gprs_1', 'gprs_2', 'cc',  'office',  'roaming',  'dishonesty',  'app', 'obsession',  'tarif',  'err_detr', 
                               'other', 'call_result', 'date_call',   'Тип оффера',  'entryid',  'sd', 'Location', 'Coordinate', 'description', 'Заявка', 'Оффер', 
                               'Реальная ТК']) 

#'qw1', 'qw2', 'qw3', 'qw4', 'qw5', 'com5', 'qw6', 'com6','voice_2', 'voice_3', 'gprs_3',
# 'gprs_comm','mark1', 'mark2', 'mark3', 'mark4', 'mark5','voice_comm','office_comm','roaming_comm',
# 'cc_comm','obsession_comm','tarif_comm','err_detr_comm','code','idinhouseservices','service','app_comm', 'dishonesty_comm', 'comp_pack',

step 2. Тянем данные из Теры

In [332]:
#NPS - подтягиваем недостающие данные
user="vitaly.flerin"
password="4GyyERMq!!!"
query = """ select t1.create_date, t1.msisdn,  t1.cluster_actual as cluster_name,  t1.macroregion, t1.region,  t1.subs_id,  t1.lt_day, t1.Lifetime,  t1.age,  t1.gender,  t1.ltr as nps, t1.nps as nps_key
 from uat_ca.v3_nps_bu t1 where (point_name = 'Контакт-центр' or point_name = 'Мобильный интернет' or point_name ='Голос и СМС')  
 and create_date between '2022-06-30' and '2023-12-31'  """
# t1.mark_6,  t1.class_mark_6,  t1.micro_class_mark_6, t1.cluster_mark_6_pr_2,  t1.servelat,, t1.tp_id, t1.tp_name, t1.tac, t1.ARPU, t1.servelat_gr,t1.cust_id,
with teradatasql.connect(host="td2800.corp.tele2.ru", user=user, password=password,logmech="ldap", tmode="TERA" ) as connect:
    bu_tera = pd.read_sql(query, connect)

C:\Users\vitaly.flerin\AppData\Local\Temp\ipykernel_5148\2400986984.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bu_tera = pd.read_sql(query, connect)


In [355]:
#Cuvo - подтягиваем данные cuvo
query_cuvo = """ select create_date, msisdn,  cuvo ,  ans_1, ans_2, ans_3
from uat_ca.v_poll_459 where create_date between '2022-06-30' and '2023-12-31'"""
#
with teradatasql.connect(host="td2800.corp.tele2.ru", user=user, password=password,logmech="ldap", tmode="TERA" ) as connect:
    cuvo = pd.read_sql(query_cuvo, connect)

C:\Users\vitaly.flerin\AppData\Local\Temp\ipykernel_5148\1344614933.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cuvo = pd.read_sql(query_cuvo, connect)


In [402]:
#меняем тип данных для объединения
bu_tera['create_date'] = pd.to_datetime(bu_tera['create_date'])
bu_tera['year'] = bu_tera['create_date'].dt.year
bu_tera['month'] = bu_tera['create_date'].dt.month
###### погуглить
#bu_tera['msisdn'] = bu_tera['msisdn'].astype(int) -
bu_tera['msisdn'] = pd.to_numeric(bu_tera['msisdn'], errors='coerce')
#bu_tera = bu_tera.dropna(subset=['msisdn']).astype({'msisdn': int})
#непосредственно объединение
merged_df = pd.merge(df_kc, bu_tera, left_on='custnum', right_on = 'msisdn', how='left', suffixes = ['_x','_y'] )
merged_df = merged_df.query('month_x == month_y or month_x + 1 == month_y')
#merged_df = merged_df.drop(columns='month_y').rename(columns={'month_x': 'month'})

In [403]:
cuvo['msisdn'] = pd.to_numeric(cuvo['msisdn'], errors='coerce')
cuvo['create_date'] = pd.to_datetime(cuvo['create_date'])
cuvo['month'] = cuvo['create_date'].dt.month
df_cuvo_nps = pd.merge(merged_df, cuvo, left_on=['custnum'], right_on = ['msisdn'] , how = 'left') 
#df_cuvo_nps= df_cuvo_nps.query('month_x == month or month_x + 1 == month or month_x + 2 == month or month.isna()')

In [404]:
# конвертируем user в 1/0
""" 
Функция func принимает одну ячейку из столбца в качестве аргумента row.
Функция проверяет, является ли значение в этой ячейке строкой и содержит ли строка только пробелы, используя метод strip(). 
Если это так, то функция возвращает 0, что обозначает отсутствие информации в данной ячейке.
Если значение в этой ячейке не содержит только пробелы, функция проверяет, является ли значение NaN, используя метод pd.isna(). 
Если значение NaN, то функция также возвращает 0, что является признаком отсутствия информации в данной ячейке.
Наконец, если значение в этой ячейке не является NaN и содержит что-то, отличное от пробелов, то функция возвращает 1, что означает наличие информации в данной ячейке.
Далее, используя функцию apply(), мы можем применить эту функцию к каждой ячейке столбца и создать новый столбец, который будет содержать 0 или 1 в зависимости от того, содержит ли ячейка информацию или нет. 
Обычно создаваемый столбец называется маркером пропусков (missing value indicator), потому что он показывает, где в исходном столбце находятся пропущенные значения. """

def func(row):
    if isinstance(row, str) and not row.strip():
        # Проверка на пустую строку
        return 0
    if pd.isna(row):
        # Проверка на NaN
        return 0
    return 1
df_cuvo_nps['call_status'] = df_cuvo_nps['user'].apply(func)
df_cuvo_nps['rtk_voice_flg'] = df_cuvo_nps['voice_1'].apply(func)
df_cuvo_nps['rtk_mi_flg'] = df_cuvo_nps['gprs_1'].apply(func)
df_cuvo_nps['rtk_ds_flg'] = df_cuvo_nps['cc'].apply(func)
df_cuvo_nps['rtk_monobrand_flg'] = df_cuvo_nps['office'].apply(func)
df_cuvo_nps['rtk_roaming_flg'] = df_cuvo_nps['roaming'].apply(func)
df_cuvo_nps['rtk_dishonesty_flg'] = df_cuvo_nps['dishonesty'].apply(func)
df_cuvo_nps['rtk_lk_flg'] = df_cuvo_nps['app'].apply(func)
df_cuvo_nps['rtk_annoying_flg'] = df_cuvo_nps['obsession'].apply(func)
df_cuvo_nps['rtk_tarif_flg'] = df_cuvo_nps['tarif'].apply(func)
df_cuvo_nps['error_detract'] = df_cuvo_nps['err_detr'].apply(func)
df_cuvo_nps['rtk_flg'] =  df_cuvo_nps['rtk_voice_flg'] + df_cuvo_nps['rtk_mi_flg'] + df_cuvo_nps['rtk_ds_flg'] + df_cuvo_nps['rtk_monobrand_flg'] + df_cuvo_nps['rtk_roaming_flg'] + df_cuvo_nps['rtk_dishonesty_flg'] + df_cuvo_nps['rtk_lk_flg'] + df_cuvo_nps['rtk_annoying_flg'] + df_cuvo_nps['rtk_tarif_flg']

In [405]:
def func_2(row):
    if isinstance(row, str) and not row.strip(): # Проверка на пустую строку
        return 0
    if pd.isna(row): # Проверка на NaN
        return 0
    if row == 0: # Проверка на наличие 0 в строке
       return 0
    return 1
df_cuvo_nps['rtk_flg_'] = df_cuvo_nps['rtk_flg'].apply(func_2)

In [406]:
#разюиваем ячейку на долготу и широту
df_cuvo_nps[['lon', 'lat']] = df_cuvo_nps['Coordinate'].str.split(' ', expand=True)
#t['lon'] = t['lon'].str.strip('(')
#t['lat'] = t['lat'].str.strip(')')

In [ ]:
pd.set_option('display.max_columns', 100) 
df_cuvo_nps

In [408]:
a=[]
for col in df_cuvo_nps.columns:
    a.append(col)
print(a)    

['custnum', 'nps_x', 'user', 'date', 'week', 'month_x', 'year_x', 'region', 'macro', 'cont_point', 'cust_comm', 'voice_1', 'gprs_1', 'gprs_2', 'cc', 'office', 'roaming', 'dishonesty', 'app', 'obsession', 'tarif', 'err_detr', 'other', 'call_result', 'date_call', 'Тип оффера', 'entryid', 'sd', 'Location', 'Coordinate', 'description', 'Заявка', 'Оффер', 'Реальная ТК', 'create_date_x', 'msisdn_x', 'cluster_name', 'Macroregion', 'Region', 'subs_id', 'lt_day', 'Lifetime', 'age', 'gender', 'nps_y', 'nps_key', 'year_y', 'month_y', 'create_date_y', 'msisdn_y', 'cuvo', 'ans_1', 'ans_2', 'ans_3', 'month', 'call_status', 'rtk_voice_flg', 'rtk_mi_flg', 'rtk_ds_flg', 'rtk_monobrand_flg', 'rtk_roaming_flg', 'rtk_dishonesty_flg', 'rtk_lk_flg', 'rtk_annoying_flg', 'rtk_tarif_flg', 'error_detract', 'rtk_flg', 'rtk_flg_', 'lon', 'lat']


In [409]:

t = df_cuvo_nps.reindex(columns=['custnum', 'nps_x', 'user', 'date', 'week', 'month_x', 'year_x', 'region', 'macro', 'cont_point', 'cust_comm', 'voice_1', 'gprs_1', 'gprs_2', 'cc', 'office',
                                 'roaming', 'dishonesty', 'app', 'obsession', 'tarif', 'err_detr', 'other', 'call_result', 'date_call', 'Тип оффера', 'entryid', 'sd', 'Location',
                                 'Coordinate', 'description', 'Заявка', 'Оффер', 'Реальная ТК', 'create_date_x', 'msisdn_x', 'cluster_name', 'Macroregion', 'Region', 'subs_id', 
                                 'lt_day', 'Lifetime', 'age', 'gender', 'nps_y', 'nps_key', 'year_y', 'month_y', 'create_date_y', 'msisdn_y', 'cuvo', 'call_status',  'ans_1', 'ans_2', 'ans_3',
                                 'rtk_voice_flg', 'rtk_mi_flg', 'rtk_ds_flg', 'rtk_monobrand_flg', 
                                 'rtk_roaming_flg', 'rtk_dishonesty_flg', 'rtk_lk_flg', 'rtk_annoying_flg', 'rtk_tarif_flg', 'error_detract', 'rtk_flg', 'rtk_flg_', 'lon', 'lat']) 

#'voice_1', 'gprs_1', 'cc', 'office', 'roaming','dishonesty', 'app', 'obsession', 'tarif', 'err_detr', 'other','Coordinate','create_date_x','msisdn_x','macro'
# ,'Region', 'nps_y', 'year_y', 'month_y','msisdn_y', 'month', 'month', 

In [417]:
#переименовываем
t = t.rename(columns = dict(custnum = 'msisdn', date = 'create_date', call_status = 'call_flg', cont_point = 'point_name', Lifetime = 'lt_gr',
                     comp_pack = 'offer_type', voice_1='rtk_voice', voice_2 = 'rtk_voice_reason', gprs_1 = 'rtk_mi', nps_y = 'nps', ARPU = 'arpu_segment' ,
                     gprs_2 = 'rtk_mi_reason', cc = 'rtk_ds', cc_comm=  'rtk_ds_reason', office = 'rtk_monobrand', office_comm = 'rtk_monobrand_reason', roaming = 'rtk_roaming', roaming_comm = 'rtk_roaming_reason',
                     dishonesty = 'rtk_dishonesty', dishonesty_comm = 'rtk_dishonesty_reason', app = 'rtk_lk', app_comm = 'rtk_lk_reason', obsession = 'rtk_annoying', obsession_comm = 'rtk_annoying_reason', 
                     tarif = 'rtk_tarif', tarif_comm = 'rtk_tarif_reason',  err_detr_comm = 'err_detr_comm' , date_call =  'call_date'))
t = t.rename(columns = {'Тип оффера': 'type_off', 'Заявка' :'application',  'Оффер': 'offer_flg', 'Реальная ТК': 'real_point_contact' })

In [412]:
t.to_excel('результат_детракторы.xlsx')

In [416]:
result = [] 
directory = '//corp.tele2.ru/msfolders/inhouse_tp'
 
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.csv'):
            result.append(os.path.join(root, file))
result  
#мапп данных 
#folder_path = '//corp.tele2.ru/msfolders/inhouse_tp\\221216'
#file_list = glob.glob(folder_path + "/*.csv")
main_dataframe = pd.DataFrame(pd.read_table(result[0],   delimiter= ";", encoding="cp1251", on_bad_lines='skip', header=None))
for i in range(1,len(result)):
    data = pd.read_table(result[i], delimiter= ";", encoding="cp1251", on_bad_lines='skip', header=None)
    df = pd.DataFrame(data)
    main_dataframe = pd.concat([main_dataframe, df], axis = 0)  
main_dataframe.reset_index(drop= True , inplace= True)  

#обработка данных
main_dataframe.columns = [
'WORKLIST_ID', 'CLIENT_ID', 'PHONE1','PHONE2','PHONE3','MSISDN','TARIFF_PLAN','SCRIPT_ID','P_INVOICE','PA_INVOICE','CUST1', 'CUST2','CUST3',
'CUST4','CUST5','CUST6','NPS','REP_DTE' ,'REGION',  'CONT_POINT','QW1', 'QW2','QW3','QW4','QW5','COM5','QW6','COM6','COMP_PACK','CUST_COMM', 
'VOICE_1','VOICE_2','VOICE_3','VOICE_COMM', 'GPRS_1' , 'GPRS_2' , 'GPRS_3', 'GPRS_COMM' ,'CC','CC_COMM' ,'OFFICE'  ,'OFFICE_COMM','ROAMING',
'ROAMING_COMM' ,'DISHONESTY' ,  'DISHONESTY_COMM' , 'APP' , 'APP_COMM' ,'OBSESSION' ,'OBSESSION_COMM' ,'TARIFF' ,'TARIFF_COMM' ,'ERR_DETR' ,
'ERR_DETR_COMM','OTHER','CALL_RESULT','TREATMENTCODE' ,'user', 'date_call','OBJ3','OBJ4','OBJ5','OBJ6','OBJ7','OBJ8','OBJ9','OBJ10','OBJ11',
'OBJ12','OBJ13','OBJ14','OBJ15','OBJ16','OBJ17' ,'OBJ18' ,'OBJ19','OBJ20','OBJ21' , 'OBJ22' , 'OBJ23' , 'OBJ24','OBJ25' , 'OBJ26' , 'OBJ27',
'OBJ28', 'OBJ29', 'OBJ30' ,  'OBJ31','OBJ32' , 'OBJ33','OBJ34','OBJ35','OBJ36','OBJ37' , 'OBJ38','OBJ39' , 'OBJ40','OBJ41','OBJ42', 'OBJ43',
'OBJ44', 'OBJ45','OBJ46','OBJ47' ,'OBJ48' , 'OBJ49'  , 'OBJ50','OBJ51','OBJ52' ,'OBJ53' , 'OBJ54' ,'OBJ55', 'OBJ56','OBJ57','OBJ58','OBJ59',
'OBJ60','OBJ61', 'OBJ62', 'OBJ62','OBJ63','OBJ64','OBJ65','OBJ66','OBJ67','OBJ68','OBJ69','OBJ70', 'OBJ71'
]

main_dataframe.drop([
'WORKLIST_ID', 'CLIENT_ID', 'PHONE1','PHONE2','PHONE3', 'TARIFF_PLAN','SCRIPT_ID','P_INVOICE','PA_INVOICE','CUST1','CUST2','CUST3','CUST4','CUST5','CUST6',
'TREATMENTCODE',  'OBJ3','OBJ4','OBJ5','OBJ6','OBJ7','OBJ8','OBJ9','OBJ10','OBJ11','OBJ12','OBJ13','OBJ14','OBJ15','OBJ16','OBJ17','OBJ18','OBJ19','OBJ20',
'OBJ21','OBJ22',  'OBJ23','OBJ24', 'OBJ25','OBJ26','OBJ27','OBJ28','OBJ29','OBJ30','OBJ31','OBJ32','OBJ33','OBJ34','OBJ35','OBJ36','OBJ37','OBJ38','OBJ39',
'OBJ40','OBJ41','OBJ42', 'OBJ43','OBJ44','OBJ45', 'OBJ46','OBJ47','OBJ48','OBJ49','OBJ50','OBJ51', 'OBJ52','OBJ53','OBJ54','OBJ55','OBJ56','OBJ57','OBJ58',
'OBJ59','OBJ60','OBJ61', 'OBJ62', 'OBJ62','OBJ63','OBJ64','OBJ65','OBJ66','OBJ67','OBJ68','OBJ69','OBJ70', 'OBJ71'
    ], axis= 1 , inplace= True )

main_dataframe = main_dataframe[[
'MSISDN', 'NPS', 'user', 'REP_DTE', 'REGION'  , 'CONT_POINT', 'QW1',   'QW2'  , 'QW3','QW4', 'QW5' ,'COM5' , 'QW6', 'COM6', 
'COMP_PACK', 'CUST_COMM', 'VOICE_1', 'VOICE_2'  , 'VOICE_3'  , 'VOICE_COMM'  , 'GPRS_1' , 'GPRS_2','GPRS_3' ,  'GPRS_COMM', 
'CC', 'CC_COMM' , 'OFFICE' , 'OFFICE_COMM', 'ROAMING', 'ROAMING_COMM', 'DISHONESTY', 'DISHONESTY_COMM', 'APP' , 'APP_COMM',
 'OBSESSION', 'OBSESSION_COMM', 'TARIFF'  , 'TARIFF_COMM', 'ERR_DETR', 'ERR_DETR_COMM','OTHER','CALL_RESULT',  'date_call'
]]
#main_dataframe
main_dataframe['date_call'] = pd.to_datetime(main_dataframe['date_call'])
main_dataframe.dropna(subset=['MSISDN'], inplace= True) 
tp = main_dataframe.copy()
tp['year'] = pd.DatetimeIndex(tp['date_call']).year
tp = tp.dropna (subset=['year'])
tp['year'] = tp['year'].astype(int)
tp
#main_dataframe.to_excel('inh_tp.xlsx', index=False)

,MSISDN,NPS,user,REP_DTE,REGION,CONT_POINT,QW1,QW2,QW3,QW4,QW5,COM5,QW6,COM6,COMP_PACK,CUST_COMM,VOICE_1,VOICE_2,VOICE_3,VOICE_COMM,GPRS_1,GPRS_2,GPRS_3,GPRS_COMM,CC,CC_COMM,OFFICE,OFFICE_COMM,ROAMING,ROAMING_COMM,DISHONESTY,DISHONESTY_COMM,APP,APP_COMM,OBSESSION,OBSESSION_COMM,TARIFF,TARIFF_COMM,ERR_DETR,ERR_DETR_COMM,OTHER,CALL_RESULT,date_call,year
0,7.904309e+10,5.0,Tsybenov,2022-12-12,Челябинск,Тарифы,NaN,"Дороже, чем у других операторов",NaN,NaN,NaN,NaN,NaN,NaN,Пакет,NaN,-,-,-,NaN,В регионе где проживает,Другое,-,в некоторых местах интернет плохо работает и п...,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,Другое,оплата по тарифу растет,-,NaN,NaN,Не решено,2022-12-13,2022
1,7.951444e+10,6.0,FrolovaK,2022-12-12,Челябинск,Тарифы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пакет,NaN,-,-,-,NaN,-,-,-,NaN,-,NaN,Мало офисов/далеко находится,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,NaN,Не решено,2022-12-14,2022
2,7.908606e+10,3.0,Tsybenov,2022-12-12,Мурманск,Тарифы,NaN,Не устраивает соотношение цена/качество,NaN,NaN,NaN,NaN,NaN,"Нет льгот для льготников, нет поощрений. А в н...",Пакет,NaN,-,-,-,NaN,В поездках по России,Зависания сбои разрывы соединения,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,Не хватает скидок/бонусов,NaN,-,NaN,NaN,Не решено,2022-12-13,2022
3,7.904396e+10,1.0,PetsAn,2022-12-12,Н.Новгород,Тарифы,NaN,Мало доп.услуг и сервисов,NaN,NaN,NaN,NaN,NaN,1,-,NaN,-,-,-,NaN,-,-,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,NaN,-,2022-12-13,2022
4,7.952372e+10,1.0,NaN,2022-12-12,С.Петербург,Тарифы,NaN,Другое,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,-,-,-,NaN,-,-,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,NaN,-,2022-12-14,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11259,7.900521e+10,3.0,Gladkih,2023-06-23,Киров,Тарифы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пакет,NaN,-,-,-,NaN,В регионе где проживает,-,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,NaN,Не решено,2023-06-24,2023
11260,7.902753e+10,4.0,Zik,2022-12-05,Тула,Тарифы,NaN,Подключили/перевели на невыгодный для меня тариф,NaN,NaN,NaN,NaN,NaN,"Условия меняются на худшие, новые тарифы не вы...",-,NaN,-,-,-,NaN,-,-,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,Не устраивает соотношение цены и наполнения та...,NaN,-,NaN,NaN,Не решено,2023-06-24,2023
11261,7.902803e+10,3.0,Gladkih,2022-04-08,Пермь,Тарифы,4.0,5,NaN,NaN,NaN,NaN,NaN,NaN,Скидка,NaN,-,-,-,NaN,-,-,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,Дороже чем у других операторов,NaN,-,NaN,NaN,Решено,2023-06-24,2023
11262,7.902509e+10,5.0,Zik,2023-06-23,Екатеринбург,Тарифы,NaN,Не устраивает соотношение цена/качество,NaN,NaN,NaN,NaN,NaN,NaN,Скидка,NaN,-,-,-,NaN,-,-,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,-,NaN,Не устраивает соотношение цены и наполнения та...,NaN,-,NaN,NaN,Решено,2023-06-24,2023


In [ ]:
#считываем данные тех лок 
result = [] 
directory = '//corp.tele2.ru/zrfolders/Report_CC/Отчеты по ТТ/TechLocation/'

for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.xlsx'):
            result.append(os.path.join(root, file))
#result  
#мапп данных 
#folder_path = '//corp.tele2.ru/msfolders/inhouse_tp\\221216'
#file_list = glob.glob(folder_path + "/*.csv")
main_dataframe = pd.DataFrame(pd.read_excel(result[1]))  #delimiter= ";", on_bad_lines='skip', header=None , encoding="cp1251"
for i in range(2,len(result)):
    data = pd.read_excel(result[i]) #, delimiter= ";", encoding="cp1251", on_bad_lines='skip', header=None , encoding="cp1251"
    df = pd.DataFrame(data)
    main_dataframe = pd.concat([main_dataframe, df], axis = 0)  
main_dataframe.reset_index(drop= True , inplace= True)  
main_dataframe.dropna(subset=['Номер абонента'], inplace= True) 

tt = main_dataframe.copy()
#main_dataframe['Номер абонента'] = main_dataframe['Номер абонента'].astype(str).str.replace('\D+', '')
#main_dataframe['Номер абонента'] = main_dataframe['Номер абонента'].astype(np.float64)
tt['Номер абонента'] = tt['Номер абонента'].astype(str).str.replace('\D+', '')
tt['Номер абонента'] = tt['Номер абонента'].replace ( r'' , np.nan , regex= True )
tt['Номер абонента'] = tt['Номер абонента'].astype(np.float64)
tt['year'] = pd.DatetimeIndex(tt['Дата создания заявки']).year
tt
#print(tt.info())
#print(tp.info())
#tt.to_excel('выгрузка_пример.xlsx')
#tt[tt["Номер заявки"] == "TT17598667"]

In [6]:
#считываем данные по МИ и КЦ
df_ds = pd.read_excel(r'\\corp.tele2.ru\ZRfolders\CrossFunctionalReports\InHouse_NPS\InHouse_NPS.xlsx')

Объеденение данных

In [7]:
#full_data = pd.merge(tp, tt, how='left', left_on=['date_call', 'MSISDN'], right_on = ['Дата создания заявки','Номер абонента']) # merge
#pd.set_option('display.max_columns', None)
full_data = pd.merge(tp, tt, how='left', left_on=['year', 'MSISDN'], right_on = ['year' ,'Номер абонента']) # merge
full_data[["pol1", "pol2", "pol3", "pol4", "pol5", "pol6", "pol7", "pol8", "pol9", "pol10", "pol11", "pol12", "pol13", "pol14", "pol15"]] = np.NaN
full_data = full_data[[
'MSISDN', 'NPS', 'user',  'REP_DTE', "pol1", "pol2", 'year', 'REGION',  "pol3", 'CONT_POINT', 'QW1', 'QW2', 'QW3', 'QW4', 'QW5', 'COM5', 
'QW6', 'COM6', 'COMP_PACK', 'CUST_COMM', 'VOICE_1', 'VOICE_2', 'VOICE_3' ,  'VOICE_COMM', 'GPRS_1',   'GPRS_2'  ,'GPRS_3',  'GPRS_COMM',
'CC',  'CC_COMM', 'OFFICE',  'OFFICE_COMM',  'ROAMING', 'ROAMING_COMM', 'DISHONESTY','DISHONESTY_COMM', 'APP', 'APP_COMM',  'OBSESSION',
 'OBSESSION_COMM',  'TARIFF', 'TARIFF_COMM', 'ERR_DETR', 'ERR_DETR_COMM', 'OTHER',   'CALL_RESULT', 'date_call', "pol4", "pol5", "pol6", 
 "pol7", "pol8",    "pol9", "pol10", "pol11", 'Номер заявки', 'Сервис',         'Краткое описание проблемы',  'Местоположение абонента', 
 'Координаты',"pol12", "pol13", "pol14", "pol15"]]
full_data.columns = ['custnum', 	'nps', 	'user', 	'date', 	'week', 	'month', 	'year', 	'region', 	'macro', 	'cont_point', 	'qw1', 	'qw2', 	'qw3', 	'qw4', 	'qw5', 	'com5', 
	'qw6', 	'com6', 	'comp_pack', 	'cust_comm', 	'voice_1', 	'voice_2', 	'voice_3', 	'voice_comm', 	'gprs_1', 	'gprs_2', 	'gprs_3', 	'gprs_comm', 	
    'cc', 	'cc_comm', 	'office', 	'office_comm', 	'roaming', 	'roaming_comm', 	'dishonesty', 	'dishonesty_comm', 	'app', 	'app_comm', 	'obsession', 	'obsession_comm', 	
    'tarif', 	'tarif_comm', 	'err_detr', 	'err_detr_comm', 	'other', 	'call_result', 	'date_call', 	'code', 	'mark1', 	'mark2', 	'mark3', 	'mark4', 	'mark5', 	'Тип оффера', 
    	'idinhouseservices', 	'entryid', 	'service', 	'sd', 	'Location', 	'Coordinate', 	'description', 	'Заявка', 	'Оффер', 	'Реальная ТК']
full_data['date'] = pd.to_datetime(full_data['date'])
full_data[['voice_1', 	'voice_2',	'voice_3', 	'voice_comm', 	'gprs_1',	'gprs_2', 	'gprs_3',	'gprs_comm', 'cc',	'cc_comm',	'office',	'office_comm',
	'roaming',	'roaming_comm',	'dishonesty',	'dishonesty_comm',	'app',	'app_comm',	'obsession',	'obsession_comm',	'tarif',	'tarif_comm',	'err_detr',
    'err_detr_comm',	'other',	'call_result']] = full_data[['voice_1', 	'voice_2',	'voice_3', 	'voice_comm', 	'gprs_1',	'gprs_2', 	'gprs_3',	'gprs_comm', 'cc',	'cc_comm',	'office',	'office_comm',
	'roaming',	'roaming_comm',	'dishonesty',	'dishonesty_comm',	'app',	'app_comm',	'obsession',	'obsession_comm',	'tarif',	'tarif_comm',	'err_detr',
    'err_detr_comm',	'other',	'call_result']].replace("-", np.NaN)
full_data
#df['workclass'].replace('?', np.NaN) 
#full_data.columns
#full_data.to_excel('выгрузка_пример_2.xlsx')

,custnum,nps,user,date,week,month,year,region,macro,cont_point,...,idinhouseservices,entryid,service,sd,Location,Coordinate,description,Заявка,Оффер,Реальная ТК
0,7.904309e+10,5.0,Tsybenov,2022-12-12,NaN,NaN,2022,Челябинск,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.951444e+10,6.0,FrolovaK,2022-12-12,NaN,NaN,2022,Челябинск,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.908606e+10,3.0,Tsybenov,2022-12-12,NaN,NaN,2022,Мурманск,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.904396e+10,1.0,PetsAn,2022-12-12,NaN,NaN,2022,Н.Новгород,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.952372e+10,1.0,NaN,2022-12-12,NaN,NaN,2022,С.Петербург,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8331,7.977848e+10,1.0,NaN,2022-12-07,NaN,NaN,2023,Москва,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8332,7.952329e+10,3.0,NaN,2023-04-13,NaN,NaN,2023,Пермь,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8333,7.977261e+10,4.0,FrolovaM,2023-04-13,NaN,NaN,2023,Москва,NaN,Тарифы,...,NaN,TT21532485,Технологическая заявка,Низкая скорость передачи данных,"улица Дружбы, 7к2, Люберцы, Московская область",55.703373 37.964484,NaN,NaN,NaN,NaN
8334,7.977261e+10,4.0,FrolovaM,2023-04-13,NaN,NaN,2023,Москва,NaN,Тарифы,...,NaN,TT20679286,Технологическая заявка,Низкая скорость передачи данных,"улица Дружбы, 7к2, Люберцы, Московская область",55.703373 37.964484,NaN,NaN,NaN,NaN


In [8]:
df_svod = pd.concat([df_ds, full_data])
df_svod.reset_index(drop= True , inplace= True )
df_svod

,custnum,nps,user,date,week,month,year,region,macro,cont_point,...,idinhouseservices,entryid,service,sd,Location,Coordinate,description,Заявка,Оффер,Реальная ТК
0,7.901333e+10,1.0,NaN,2023-04-14,15.0,4.0,2023,Москва,Москва,Мобильный интернет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,7.950833e+10,1.0,NaN,2023-04-14,15.0,4.0,2023,Ижевск,Волга,Мобильный интернет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,7.902797e+10,1.0,NaN,2023-04-14,15.0,4.0,2023,Пермь,Урал,Мобильный интернет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
3,7.992305e+10,1.0,NaN,2023-04-14,15.0,4.0,2023,Тюмень,Урал,Мобильный интернет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
4,7.904444e+10,1.0,NaN,2023-04-14,15.0,4.0,2023,Ростов На дону,Юг,Мобильный интернет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336529,7.977848e+10,1.0,NaN,2022-12-07,NaN,NaN,2023,Москва,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336530,7.952329e+10,3.0,NaN,2023-04-13,NaN,NaN,2023,Пермь,NaN,Тарифы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336531,7.977261e+10,4.0,FrolovaM,2023-04-13,NaN,NaN,2023,Москва,NaN,Тарифы,...,NaN,TT21532485,Технологическая заявка,Низкая скорость передачи данных,"улица Дружбы, 7к2, Люберцы, Московская область",55.703373 37.964484,NaN,NaN,NaN,NaN
336532,7.977261e+10,4.0,FrolovaM,2023-04-13,NaN,NaN,2023,Москва,NaN,Тарифы,...,NaN,TT20679286,Технологическая заявка,Низкая скорость передачи данных,"улица Дружбы, 7к2, Люберцы, Московская область",55.703373 37.964484,NaN,NaN,NaN,NaN


In [ ]:
df_svod['date'] = pd.to_datetime(df_svod['date']). dt.date
df_svod['date_call'] = pd.to_datetime(df_svod['date_call']). dt.date
df_svod.info()

In [10]:
#форматы даты, что - то придумать
df_svod.to_excel('qwe.xlsx', index=False)

In [20]:
for col in list(df_svod):
    df_svod[col] = df_svod[col].astype(str)
    df_svod[col].replace('nan', None, inplace = True)

In [26]:
df_list = []
for i in df_svod.index:
    df_list.append([df_svod.at[i, _] for _ in list(df_svod)])

In [22]:
#print(df_list[ : 5])

[['79515813050.0', '1.0', None, '2023-02-17', '7.0', '2.0', '2023', 'Кемерово', 'Сибирь', 'Контакт-центр', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2023-02-20', '38.0', None, None, None, None, None, None, None, None, None, None, None, None, None, '0.0', '0.0', None], ['79041130030.0', '1.0', None, '2023-02-17', '7.0', '2.0', '2023', 'Иркутск', 'Байкал и Дальний Восток', 'Контакт-центр', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2023-02-20', '38.0', None, None, None, None, None, None, None, None, None, None, None, None, None, '0.0', '0.0', None], ['79535868006.0', '1.0', None, '2023-02-17', '7.0', '2.0', '2023', 'Красноярск', 'Сибирь', 'Конта

In [ ]:
with teradatasql.connect(host = "td2800.corp.tele2.ru", user = user, password = password, logmech = 'LDAP') as connect:
        cursor = connect.cursor()
        cursor.executemany('insert into uat_ca.vf_inh_tset (msisdn, nps, "user", create_date, "week", "month", "year", region, "macro", point_name, qw1, qw2, qw3, qw4, qw5, com5, qw6,	com6, offer_type,	cust_comm,	rtk_voice,	rtk_voice_reason,	voice_3,	voice_comm,	rtk_mi,	rtk_mi_reason,	gprs_3,	gprs_comm,	rtk_ds,	rtk_ds_reason,	rtk_monobrand,	rtk_monobrand_reason,	rtk_roaming,	rtk_roaming_reason,	rtk_dishonesty,	rtk_dishonesty_reason,	rtk_lk,	rtk_lk_reason,	rtk_annoying,	rtk_annoying_reason,	rtk_tarif,	rtk_tarif_reason,	error_detract,	err_detr_comm,	other,	call_status,	call_date,	code,	mark1,	mark2,	mark3,	mark4,	mark5,	type_off,	idinhouseservices,	entryid,	service,	sd,	Location,	Coordinate,	description,	application,	offer_flg, real_point_contact) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);', df_list)
        connect.commit() 
        

In [ ]:
df_list[51569]

In [ ]:
df_list[51568]

In [10]:
user="vitaly.flerin"
password="4GyyERMq!!"


черновик

In [ ]:
df_svod.to_excel('wer.xlsx')

In [ ]:
# тест
main_dataframe = pd.DataFrame(pd.read_table(result[0],  sep=';',  encoding="cp1251", on_bad_lines='skip'))
main_dataframe

In [40]:
#tt['Номер абонента_new'] = tt['Номер абонента'].str.replace("+", "")
#tt['Номер абонента_new'] = tt['Номер абонента'].str.replace("-", "")
#tt['Номер абонента'] = tt['Номер абонента']. replace(['+', '-'], ['', ''])
#tt['result'] = tt['Номер абонента'].map(lambda x: x.lstrip('+-'))

tt